# Set-up

In [2]:
# imports
import os
import sys
import synapseclient

In [3]:
path_out = "/cellar/users/aklie/data/datasets/tf_perturb_seq/datasets/Huangfu_WTC11-benchmark_TF-Perturb-seq/results/1_CRISPR_pipeline/2025_11_26"

# Login

In [4]:
# Token
SYNAPSE_AUTH_TOKEN = os.environ.get('SYNAPSE_AUTH_TOKEN')

In [5]:
# Login
syn = synapseclient.Synapse() 
syn.login(authToken=SYNAPSE_AUTH_TOKEN)


UPGRADE AVAILABLE

A more recent version of the Synapse Client (4.10.0) is available. Your version (2.7.2) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 4.10.0 release notes

https://python-docs.synapse.org/news/



Welcome, Adam Klie!



# Upload

In [24]:
project_id = "syn64423137"

In [25]:
datasets = [
    "Gersbach_WTC11-benchmark_TF-Perturb-seq_GEM-Xv3",
    "Gersbach_WTC11-benchmark_TF-Perturb-seq_HTv2",
    "Hon_WTC11-benchmark_TF-Perturb-seq",
    "Huangfu_WTC11-benchmark_TF-Perturb-seq",
]

base = "/cellar/users/aklie/data/datasets/tf_perturb_seq/datasets"


def get_children_map(parent_id):
    """Return {name: id} for direct children of a Synapse folder/project."""
    return {c["name"]: c["id"] for c in syn.getChildren(parent_id)}


def get_or_create_folder(parent_id, name):
    """Get a folder by name under parent_id, or create it if it doesn't exist."""
    children = get_children_map(parent_id)
    if name in children:
        return syn.get(children[name])
    return syn.store(synapseclient.Folder(name=name, parent=parent_id))


def mirror_to_synapse(local_root, syn_parent):
    """
    Recursively mirror local_root directory structure into Synapse folder syn_parent.
    local_root itself is *not* created as a folder; its contents are.
    """
    folder_map = {local_root: syn_parent}

    for dirpath, dirnames, filenames in os.walk(local_root):
        parent_id = folder_map[dirpath]

        # Create subfolders
        for d in dirnames:
            local_subdir = os.path.join(dirpath, d)
            syn_subfolder = syn.store(
                synapseclient.Folder(name=d, parent=parent_id)
            )
            folder_map[local_subdir] = syn_subfolder["id"]

        # Upload files
        for fname in filenames:
            local_file = os.path.join(dirpath, fname)
            syn_file = synapseclient.File(path=local_file, parent=parent_id)
            syn.store(syn_file)


# ---- Map existing top-level dataset folders under the project ----
top_level_children = get_children_map(project_id)

for dataset in datasets:
    dataset_dir = os.path.join(base, dataset)
    qc_dir = os.path.join(dataset_dir, "results", "2_qc")

    if not os.path.isdir(qc_dir):
        print(f"Skipping {dataset}: local qc directory missing at {qc_dir}")
        continue

    if dataset not in top_level_children:
        print(f"Skipping {dataset}: no matching folder under {project_id} on Synapse")
        continue

    dataset_syn_id = top_level_children[dataset]
    print(f"Uploading QC tree for dataset: {dataset} -> Synapse folder {dataset_syn_id}")

    # Ensure results/ and 2_qc/ folders exist under the dataset folder
    results_folder = get_or_create_folder(dataset_syn_id, "results")
    qc_folder = get_or_create_folder(results_folder["id"], "2_qc")

    # Mirror contents of local results/2_qc into Synapse results/2_qc
    mirror_to_synapse(qc_dir, qc_folder["id"])

    print(f"Finished uploading {dataset}")

Uploading QC tree for dataset: Gersbach_WTC11-benchmark_TF-Perturb-seq_GEM-Xv3 -> Synapse folder syn70110613

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################

In [28]:
dataset_name = "technology-benchmark_WTC11_TF-Perturb-seq"
local_results_dir = (
    "/cellar/users/aklie/data/datasets/tf_perturb_seq/datasets/"
    "technology-benchmark_WTC11_TF-Perturb-seq/results"
)


def get_children_map(parent_id):
    """Return {name: id} for direct child folders of a Synapse container."""
    return {c["name"]: c["id"] for c in syn.getChildren(parent_id)}


def get_or_create_folder(parent_id, name):
    """Get a Synapse folder by name, or create it under parent_id."""
    children = get_children_map(parent_id)
    if name in children:
        return syn.get(children[name])
    return syn.store(synapseclient.Folder(name=name, parent=parent_id))


def mirror_to_synapse(local_root, syn_parent):
    """
    Recursively mirror the entire local directory tree starting at local_root
    *into* Synapse under syn_parent.
    """
    folder_map = {local_root: syn_parent}

    for dirpath, dirnames, filenames in os.walk(local_root):
        parent_id = folder_map[dirpath]

        # Create folders
        for d in dirnames:
            local_subdir = os.path.join(dirpath, d)
            syn_subfolder = syn.store(
                synapseclient.Folder(name=d, parent=parent_id)
            )
            folder_map[local_subdir] = syn_subfolder["id"]

        # Upload files
        for fname in filenames:
            file_path = os.path.join(dirpath, fname)
            syn_file = synapseclient.File(path=file_path, parent=parent_id)
            syn.store(syn_file)


# ============================ RUN UPLOAD ============================

# Ensure the dataset folder exists at the project top level
top_children = get_children_map(project_id)

if dataset_name not in top_children:
    raise ValueError(
        f"No folder named '{dataset_name}' under project {project_id}. "
        "Create it manually or correct the name."
    )

dataset_syn_id = top_children[dataset_name]

# Create or reuse dataset/results folder
results_folder = get_or_create_folder(dataset_syn_id, "results")

# Mirror the entire local results tree
print(f"Uploading entire results tree from {local_results_dir}")
mirror_to_synapse(local_results_dir, results_folder["id"])
print("Upload complete!")

Uploading entire results tree from /cellar/users/aklie/data/datasets/tf_perturb_seq/datasets/technology-benchmark_WTC11_TF-Perturb-seq/results

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


################